<a href="https://colab.research.google.com/github/Sun-Joong/aifh/blob/master/CNN_using_CIFAR10_my_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.datasets.cifar10 import load_data
import matplotlib.pyplot as plt

c:\users\user\anaconda3\envs\dl_13\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\user\anaconda3\envs\dl_13\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\user\anaconda3\envs\dl_13\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\user\anaconda3\envs\dl_13\lib\site-packages\tensorflow\python\frame

In [ ]:
epochs = 10
batch_size = 50
learning_rate = 0.001

In [ ]:
(x_train, y_train), (x_test, y_test) = load_data()

In [ ]:
x_train = x_train.astype('float32')

In [ ]:
y_train = y_train.astype('float32')

In [ ]:
x_test = x_test.astype('float32')

In [ ]:
y_test = y_test.astype('float32')

In [ ]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)

In [ ]:
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

In [ ]:
mean=[0,0,0]
std=[0,0,0]
newX_train = np.ones(x_train.shape)
newX_test = np.ones(x_test.shape)
#train set에 있는 데이터로만 평균과 표준편차를 구함
for i in range(3):
    mean[i] = np.mean(x_train[:,:,:,i])
    std[i] = np.std(x_train[:,:,:,i])

#train과 test셋 모두 정규화 작업    
for i in range(3):
    newX_train[:,:,:,i] = x_train[:,:,:,i] - mean[i]
    newX_train[:,:,:,i] = newX_train[:,:,:,i] / std[i]
    newX_test[:,:,:,i] = x_test[:,:,:,i] - mean[i]
    newX_test[:,:,:,i] = newX_test[:,:,:,i] / std[i]
        
x_train = newX_train
x_test = newX_test

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices(({"image":x_train}, y_train))

In [ ]:
dataset = dataset.shuffle(100000).repeat().batch(batch_size)
iterator = dataset.make_one_shot_iterator()
next_batch = iterator.get_next()

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


In [ ]:
class CNN_Model:
    def __init__(self, sess, name ):
        self.sess = sess
        self.name = name
        self._build_net()
        
    def _build_net(self):
        with tf.variable_scope(self.name):
            self.training = tf.placeholder(tf.bool)
            
            self.X = tf.placeholder(tf.float32, [None, 32,32,3])
            self.Y = tf.placeholder(tf.float32, [None, 10])
            
            
            conv1 = tf.layers.conv2d(inputs = self.X, filters=64, kernel_size=[3,3],padding="SAME", activation=tf.nn.relu)
            pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2,2], strides=2,padding="SAME") # 2*2 중의 가장 큰 값,stride 2*2
            dropout1 = tf.layers.dropout(inputs=pool1, rate=0.5, training=self.training)
            
            conv2 = tf.layers.conv2d(inputs = dropout1, filters=128, kernel_size=[3,3],padding="SAME", activation=tf.nn.relu)
            pool2 = tf.layers.max_pooling2d(conv2, pool_size=[2,2], strides=2,padding="SAME") # 2*2 중의 가장 큰 값,stride 2*2
            dropout2 = tf.layers.dropout(inputs=pool2, rate=0.5, training=self.training)
            
            conv3 = tf.layers.conv2d(inputs = dropout2, filters=256, kernel_size=[3,3],padding="SAME", activation=tf.nn.relu)
            pool3 = tf.layers.max_pooling2d(conv3, pool_size=[2,2], strides=2,padding="SAME") # 2*2 중의 가장 큰 값,stride 2*2
            dropout3 = tf.layers.dropout(inputs=pool3, rate=0.5, training=self.training)

            conv4 = tf.layers.conv2d(inputs = dropout3, filters=512, kernel_size=[3,3],padding="SAME", activation=tf.nn.relu)
            pool4 = tf.layers.max_pooling2d(conv4, pool_size=[2,2], strides=2,padding="SAME") # 2*2 중의 가장 큰 값,stride 2*2
            dropout4 = tf.layers.dropout(inputs=pool4, rate=0.5, training=self.training)
            
            flat = tf.reshape(dropout4, [-1, 2*2*512])
            dense5 = tf.layers.dense(inputs=flat, units=375, activation=tf.nn.relu)
            dropout5 = tf.layers.dropout(inputs=dense5, rate=0.6, training=self.training)
            
            dense6 = tf.layers.dense(inputs=dropout5, units=125, activation=tf.nn.relu)
            dropout6 = tf.layers.dropout(inputs=dense6, rate=0.6, training=self.training)
            
            self.logits = tf.layers.dense(inputs=dropout6, units=10)
            
            
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=self.logits, labels=self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.cost)

        correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
 
    def predict(self, x_test, training=False):
        return self.sess.run(self.logits,feed_dict={self.X: x_test, self.training: training})

    def get_accuracy(self, x_test, y_test, training=False):
        return self.sess.run(self.accuracy,feed_dict={self.X: x_test,self.Y: y_test, self.training: training})

    def train(self, x_train, y_train,training=True):
        return self.sess.run([self.cost, self.optimizer], feed_dict={self.X: x_train, self.Y: y_train, self.training: training}) 

In [ ]:
sess = tf.Session()
m1 = CNN_Model(sess, "m1")



Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
Instructions for updating:
Use keras.layers.dropout instead.


Instructions for updating:
Use keras.layers.dense instead.


In [ ]:
sess.run(tf.global_variables_initializer())

In [ ]:
for epoch in range(epochs):
    avg_cost = 0
    total_batch = int(len(x_train) / batch_size)

    for i in range(total_batch):
        batch_xs, batch_ys = sess.run(next_batch)
        c, _ = m1.train(batch_xs['image'], batch_ys)
        avg_cost += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning Finished!')

# Test model and check accuracy
print('Accuracy:', m1.get_accuracy(x_test, y_test))